In [ ]:
!pip install torch==1.8.2 torchvision==0.9.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu
!pip install tensorflow numpy yolov5

In [ ]:
import glob
import os
import pathlib
import shutil

import numpy as np
import tensorflow as tf

TARGET_DIR = 'train_image'

TRAIN_IMAGE = 'train_image'
VAL_IMAGE = 'val_image'
TRAIN_DIR = 'train_yolo5'
VAL_DIR = 'val_yolo5'
TRAIN_LABEL_DIR = 'train_yolo5/labels'
VAL_LABEL_DIR = 'val_yolo5/labels'
TRAIN_IMAGE_DIR = 'train_yolo5/images'
VAL_IMAGE_DIR = 'val_yolo5/images'

# 画像ファイルの取得

In [ ]:
!curl -o asset.zip -L https://d1.awsstatic.com/webteam/architecture-icons/q2-2022/Asset-Package_04302022.e942f826cd826cfa2d32455f3a7973ad4b92eb6a.zip
!unzip asset.zip -d asset

# 画像の収集と水増し

In [ ]:
def collect_target_image():

    image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        zoom_range=[0.9, 1.1],
        # rotation_range=2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        # channel_shift_range=10.0,
        fill_mode='constant',
        cval=255
    )

    os.mkdir(TARGET_DIR)
    
    files = glob.glob(
        f'asset/Architecture-Service-Icons_*/**/*64.png', recursive=True)

    for file in files:
        f = pathlib.Path(file)
        f.name
        name = f.stem.replace(
            'Arch_AWS-', '').replace('Arch_Amazon-', '').replace('Arch_', '').replace('_64', '')

        original_dir = f'{TARGET_DIR}/{name}'
        original_file = f'{original_dir}/{name}.png'

        try:
            os.mkdir(original_dir)
            shutil.copyfile(f, original_file)
        except:
            pass

        target_img = tf.keras.preprocessing.image.load_img(original_file)
        target_img = np.array(target_img)
        x = target_img.reshape((1,) + target_img.shape)

        generator = image_generator.flow(x,
                                         save_to_dir=original_dir,
                                         save_prefix=name,
                                         save_format='png')

        for i in range(100):
            generator.next()


collect_target_image()

# 学習データと検証データの分割

In [ ]:
def split_val():
    os.mkdir(VAL_IMAGE)
    dirs = os.listdir(TRAIN_IMAGE)

    for dir in dirs:
        os.mkdir(f'{VAL_IMAGE}/{dir}')

        files = os.listdir(f'{TRAIN_IMAGE}/{dir}')
        
        for file in files[:10]:            
            shutil.move(f'{TRAIN_IMAGE}/{dir}/{file}', f'{VAL_IMAGE}/{dir}/')


split_val()

# ラベルの作成

In [ ]:
def create_label():    
    os.mkdir(TRAIN_DIR)
    os.mkdir(VAL_DIR)
    os.mkdir(TRAIN_LABEL_DIR)
    os.mkdir(VAL_LABEL_DIR)

    dirs = sorted(os.listdir(TRAIN_IMAGE))

    for i, dir in enumerate(dirs):
        train_files = os.listdir(f'{TRAIN_IMAGE}/{dir}')

        for file in train_files:
            with open(f'{TRAIN_LABEL_DIR}/{file[:-4]}.txt', 'w') as f:
                f.write(f'{i} 0.5 0.5 1.0 1.0')

        val_files = os.listdir(f'{VAL_IMAGE}/{dir}')

        for file in val_files:
            with open(f'{VAL_LABEL_DIR}/{file[:-4]}.txt', 'w') as f:
                f.write(f'{i} 0.5 0.5 1.0 1.0')


create_label()

# フォルダー構成の変更

In [ ]:
def move_image():
    os.mkdir(TRAIN_IMAGE_DIR)
    os.mkdir(VAL_IMAGE_DIR)

    dirs = os.listdir(TRAIN_IMAGE)

    for dir in dirs:
        files = os.listdir(f'{TRAIN_IMAGE}/{dir}')
        
        for file in files:
            shutil.move(f'{TRAIN_IMAGE}/{dir}/{file}', f'{TRAIN_IMAGE_DIR}/')
    
    dirs = os.listdir(VAL_IMAGE)

    for dir in dirs:
        files = os.listdir(f'{VAL_IMAGE}/{dir}')
        
        for file in files:
            shutil.move(f'{VAL_IMAGE}/{dir}/{file}', f'{VAL_IMAGE_DIR}/')


move_image()

# data.yamlの作成

In [ ]:
def create_datayaml():
    dirs = sorted(os.listdir(TRAIN_IMAGE))
    cwd = os.getcwd()

    with open(f'data.yaml', 'w') as f:
        f.write(f'train: {cwd}/train_yolo5/images\n')
        f.write(f'val: {cwd}/val_yolo5/images\n')
        f.write('\n')
        f.write(f'nc: {len(dirs)}\n')
        f.write(f'names: {str(dirs)}\n')


create_datayaml()    

# 学習

In [ ]:
!yolov5 train --data data.yaml  --imgsz 80 --batch-size 8 --epochs 1

# 推論

In [ ]:
!yolov5 detect --weights runs/train/exp/weights/best.pt --source input/